# This code will be hidden from actual users within a utility class

In [1]:
import json
import requests
import urllib
import copy

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display

In [3]:
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io/legacy'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [4]:
def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def submit_to_ciars(m):
    return submit_to_ars(m,ars_url='https://ars.ci.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_ciars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars.ci.transltr.io/ars/api')


In [5]:
def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)

In [6]:
def parse(json):
    keylist = list(json.keys())
    for key in keylist:
        if key in KEYS:
            if (isinstance(json[KEYS.get(key)],dict)):
                parse(json[key])
        elif (isinstance(json[key],dict)):
            parse(json[key])

In [7]:
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())

In [8]:
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

In [9]:
def keys_exist(element, *keys):
    if not isinstance(element, dict):
        raise AttributeError('keys_exists() expects dict as first argument.')
    if len(keys) == 0:
        raise AttributeError('keys_exists() expects at least two arguments, one given.')

    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return False
        except KeyError:
            return False
    return True

def get_safe(element,*keys):
    '''
    :param element: JSON to be processed
    :param keys: list of keys in order to be traversed. e.g. "fields","data","message","results
    :return: the value of the terminal key if present or None if not
    '''
    if element is None:
        return None
    _element = element
    for key in keys:
        try:
            _element = _element[key]
            if _element is None:
                return None
            if key == keys[-1]:
                return _element
        except KeyError:
            return None
    return None

In [59]:
'''
This pulls approved drugs by pulling chembl_max_phase attribute from nodes.  It should be noted that this will not
work with drugs for which this field is not available or from Translator tools that do not implement it.
'''

def get_approved(results):
    approved = []
    for r in results:
        nodes = get_safe(result[r],"message","knowledge_graph","nodes")
        if nodes is not None:
            for node in nodes:
                attributes = get_safe(nodes[node],"attributes")
                if attributes is not None:
                    for attribute in attributes:
                        if("attribute_type_id" in attribute):
                            #print(attribute)
                            if(attribute["attribute_type_id"]=="chembl_max_phase"):
                                if("4" in attribute["value"]):
                                    approved.append(node)
                                else:
                                    print(attribute["value"])
                            
    return approved

In [11]:
def synonymize(ids, url="https://nodenormalization-sri.renci.org/1.1/get_normalized_nodes"):
        if( isinstance(ids,str) ):
            ids = [ids]
        params = {"curie":ids}
        url_curies = urllib.parse.urlencode(params, True)
        syn_url=url+"?"+url_curies
        syn_response=requests.get(syn_url)
        synonyms= syn_response.json()
        return synonyms
        
                    

In [12]:
def create_synonym_table(messages):
    synonyms={}
    for message in messages.values():
        nodes = get_safe(message,"message","knowledge_graph","nodes")
        if nodes is not None:
            sym = synonymize(nodes)
            if(len(synonyms)==0):
                synonyms= sym
            else:
                absent = []
                for node in nodes:
                    if(node in synonyms.items()):
                        continue
                    else:
                        absent.append(node)
                absent_syms = synonymize(absent)
                synonyms.update(absent_syms)
    return synonyms
                


In [13]:
def create_synonym_tables(messages):
    synonyms={}
    for entry in messages:
        actor = entry
        message = messages[actor]
        nodes = get_safe(message,"message","knowledge_graph","nodes")
        if nodes is not None:
            sym = synonymize(nodes)
            synonyms[actor]=sym
    return synonyms
                

In [14]:
def canonical_id_lists(synonym_tables):
    pks ={}
    for entry in synonym_tables:
        pks[entry]=[]
        for k,v in synonym_tables[entry].items():
            if v is None:
                pks[entry].append(k)
            else:
                canonical_id=v["id"]["identifier"]
                if (canonical_id not in pks[entry]):
                    pks[entry].append(canonical_id)
    return pks

In [15]:
def get_consensus_nodes(canonical_id_lists):
    consensus={}
    canon = copy.deepcopy(canonical_id_lists)
    my_keys= canon.keys()
    my_values=canon.values()
    for actor, id_list in canonical_id_lists.items():
        for actor2, id_list2 in canon.items():
            #Surely, there is a better way to do this with slices or something
            if(actor==actor2):
                continue
            else:
                s1 = set(id_list)
                s2 = set(id_list2)
                inter = (s1 & s2)
                if (inter):
                    for node in inter:
                        if node in consensus.keys():
                            consensus[node].add(actor)
                            consensus[node].add(actor2)
                        else:
                            consensus[node]={actor,actor2}      
        canon.pop(actor,None)
    return consensus
        

In [16]:
def get_name_from_id(curies,url="https://nodenormalization-sri.renci.org/1.1/get_normalized_nodes"):
        if( isinstance(curies,str) ):
            curies = [curies]
        params = {"curie":curies}
        url_curies = urllib.parse.urlencode(params, True)
        syn_url=url+"?"+url_curies
        syn_response=requests.get(syn_url)
        synonyms= syn_response.json()
        names={}
        for curie in curies:
            name = get_safe(synonyms,curie,"id","label")
            if name is not None:
                names[name]=curie
        return names
        

# Non-Code Nerds Start Here:

# KCNMA1 

***This query came to us by way of UAB's Precision Medicine Institute.  A mutation of the KCNMA1 gene in the patient caused paroxysmal nonkinesigenic dsykinesia (drop attacks).  Knowing that these symptoms were likely the result of the KCNMA1 mutation, we look for drugs which might have an effect on that gene or its product.***

***The below query is essentially looking for chemical substances (including but not limited to approved drugs) which might have some effect on KCNMA1 (as defined by the identifier NCBIGene:23221)***

In [26]:
kcq={
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:3778"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
		             "categories":[
                         "biolink:ChemicalSubstance"
                        ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1"

                }
            }
        }
    }
}
  


***Here, we submit the query to the ARS, which then sends it to all the connected Translator services and stores the results that are returned.***

In [27]:
kcresult = submit_to_devars(kcq)

https://arax.ncats.io/?source=ARS&id=93f89c50-728f-4033-9e5f-aa6c2031962d


***Here, we retrieve those results and display the number for each of the providers.  We may need to wait a few minutes before running this section to give the Translator tools time to compute the results***

In [28]:
result = retrieve_devars_results(kcresult)

Running
Running
Running ara-aragorn 0
Done
Done ara-arax 153
Done
Done ara-bte 138
Done
Done ara-unsecret 53
Done
Done kp-genetics 0
Error
Error kp-molecular 0
Done
Done ara-explanatory 0
Done
Done ara-improving 5
Error
Error kp-cam 0
Done
Done kp-textmining 138
Done
Done kp-openpredict 0
Done
ARS Error kp-cohd 0
Done
Done kp-chp 0
Done
Done kp-icees 0
Done
Done kp-icees-dili 0


***In the following sections, we take our results from above and sort them by the number of different tools that returned the same answer (after harmonizing all the different identifiers used).  So, the results that occur with the most concensus are at the top of the list.  The result that proved effective in the clinical setting was Vyvanse.  In our parlance, we return Methamphetamines.***

In [29]:
syns = create_synonym_tables(result)

In [30]:
canon = canonical_id_lists(syns)
con = get_consensus_nodes(canon)       
print (len(con))

99


In [31]:
for k in sorted(con,key=lambda k: len(con[k]),reverse=True):
    print(get_name_from_id(k), con[k])

{'Chlorzoxazone': 'PUBCHEM.COMPOUND:2733'} {'ara-bte', 'ara-arax', 'kp-textmining', 'ara-unsecret'}
{} {'ara-bte', 'ara-arax', 'kp-textmining', 'ara-unsecret'}
{'KCNMA1': 'NCBIGene:3778'} {'ara-bte', 'kp-textmining', 'ara-improving', 'ara-unsecret'}
{'Diazoxide': 'PUBCHEM.COMPOUND:3019'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'Linoleic acid': 'PUBCHEM.COMPOUND:5280450'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'4-Aminophenol': 'PUBCHEM.COMPOUND:403'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'glycerophosphoethanolamine': 'CHEBI:36314'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'Pinacidil': 'PUBCHEM.COMPOUND:4826'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'Ether': 'PUBCHEM.COMPOUND:3283'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'Cholesterol': 'PUBCHEM.COMPOUND:5997'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'organic sulfate': 'CHEBI:25704'} {'ara-bte', 'ara-arax', 'kp-textmining'}
{'Halothane': 'PUBCHEM.COMPOUND:3562'} {'ara-bte', 'ara



***This method should give us only the drugs which are currently FDA approved, greatly reducing the number of results
we need to consider.***



In [58]:
approved_list = get_approved(result)

In [61]:
for drug in approved_list:
    print(get_name_from_id(drug))

{'Trichlormethiazide': 'CHEMBL.COMPOUND:CHEMBL1054'}
{'Cholesterol': 'CHEMBL.COMPOUND:CHEMBL112570'}
{'CISPLATIN': 'CHEMBL.COMPOUND:CHEMBL11359'}
{'Pinacidil': 'CHEMBL.COMPOUND:CHEMBL1159'}
{'Chlorothiazide sodium': 'CHEMBL.COMPOUND:CHEMBL1200616'}
{'PROCAINE HYDROCHLORIDE': 'CHEMBL.COMPOUND:CHEMBL1200841'}
{'Benzthiazide': 'CHEMBL.COMPOUND:CHEMBL1201039'}
{'Methamphetamine': 'CHEMBL.COMPOUND:CHEMBL1201201'}
{'POLYESTRADIOL PHOSPHATE': 'CHEMBL.COMPOUND:CHEMBL1201477'}
{'Dequalinium chloride': 'CHEMBL.COMPOUND:CHEMBL121663'}
{'Enflurane': 'CHEMBL.COMPOUND:CHEMBL1257'}
{'Chlorzoxazone': 'CHEMBL.COMPOUND:CHEMBL1371'}
{'Cortisone': 'CHEMBL.COMPOUND:CHEMBL1499'}
{'Hydrocortisone cypionate': 'CHEMBL.COMPOUND:CHEMBL1549'}
{'Bendroflumethiazide': 'CHEMBL.COMPOUND:CHEMBL1684'}
{'l-Isoprenaline hydrochloride': 'CHEMBL.COMPOUND:CHEMBL1711'}
{'Hydroflumethiazide': 'CHEMBL.COMPOUND:CHEMBL1763'}
{'Diazoxide': 'CHEMBL.COMPOUND:CHEMBL181'}
{'POLYVINYL ALCOHOL': 'CHEMBL.COMPOUND:CHEMBL3559671'}
{'POTAS